# T1505.004 - IIS Components
Adversaries may install malicious components that run on Internet Information Services (IIS) web servers to establish persistence. IIS provides several mechanisms to extend the functionality of the web servers. For example, Internet Server Application Programming Interface (ISAPI) extensions and filters can be installed to examine and/or modify incoming and outgoing IIS web requests. Extensions and filters are deployed as DLL files that export three functions: <code>Get{Extension/Filter}Version</code>, <code>Http{Extension/Filter}Proc</code>, and (optionally) <code>Terminate{Extension/Filter}</code>. IIS modules may also be installed to extend IIS web servers.(Citation: Microsoft ISAPI Extension Overview 2017)(Citation: Microsoft ISAPI Filter Overview 2017)(Citation: IIS Backdoor 2011)(Citation: Trustwave IIS Module 2013)

Adversaries may install malicious ISAPI extensions and filters to observe and/or modify traffic, execute commands on compromised machines, or proxy command and control traffic. ISAPI extensions and filters may have access to all IIS web requests and responses. For example, an adversary may abuse these mechanisms to modify HTTP responses in order to distribute malicious commands/content to previously comprised hosts.(Citation: Microsoft ISAPI Filter Overview 2017)(Citation: Microsoft ISAPI Extension Overview 2017)(Citation: Microsoft ISAPI Extension All Incoming 2017)(Citation: Dell TG-3390)(Citation: Trustwave IIS Module 2013)(Citation: MMPC ISAPI Filter 2012)

Adversaries may also install malicious IIS modules to observe and/or modify traffic. IIS 7.0 introduced modules that provide the same unrestricted access to HTTP requests and responses as ISAPI extensions and filters. IIS modules can be written as a DLL that exports <code>RegisterModule</code>, or as a .NET application that interfaces with ASP.NET APIs to access IIS HTTP requests.(Citation: Microsoft IIS Modules Overview 2007)(Citation: Trustwave IIS Module 2013)(Citation: ESET IIS Malware 2021)

## Atomic Tests

### Atomic Test #1 - Install IIS Module using AppCmd.exe
The following Atomic will utilize AppCmd.exe to install a new IIS Module. IIS must be installed.
This atomic utilizes a DLL on disk, but to test further suspiciousness, compile and load [IIS-Raid](https://www.mdsec.co.uk/2020/02/iis-raid-backdooring-iis-using-native-modules/).
A successful execution will install a module into IIS using AppCmd.exe.
[Managing and installing Modules Reference](https://learn.microsoft.com/en-us/iis/get-started/introduction-to-iis/iis-modules-overview#to-install-a-module-using-appcmdexe)
[IIS Modules](https://www.microsoft.com/en-us/security/blog/2022/12/12/iis-modules-the-evolution-of-web-shells-and-how-to-detect-them/)

**Supported Platforms:** windows
#### Dependencies:  Run with `powershell`!
##### Description: IIS must be installed in order to add a module to IIS.

##### Check Prereq Commands:
```cmd
$service = get-service w3svc -ErrorAction SilentlyContinue
if($service){ Write-Host "IIS installed on $env:computername" } else { Write-Host "IIS is not installed on $env:computername" } 

```
##### Get Prereq Commands:
```cmd
Install IIS to continue.

```

In [ ]:
Invoke-AtomicTest T1505.004 -TestNumbers 1 -GetPreReqs

#### Attack Commands: Run with `command_prompt`
```command_prompt
%windir%\system32\inetsrv\appcmd.exe install module /name:DefaultDocumentModule_Atomic /image:%windir%\system32\inetsrv\defdoc.dll
```

In [ ]:
Invoke-AtomicTest T1505.004 -TestNumbers 1

#### Cleanup: 
```cmd
%windir%\system32\inetsrv\appcmd.exe uninstall module DefaultDocumentModule_Atomic
```

In [ ]:
Invoke-AtomicTest T1505.004 -TestNumbers 1 -Cleanup

### Atomic Test #2 - Install IIS Module using PowerShell Cmdlet New-WebGlobalModule
The following Atomic will utilize PowerShell Cmdlet New-WebGlobalModule to install a new IIS Module. IIS must be installed.
This atomic utilizes a DLL on disk, but to test further suspiciousness, compile and load [IIS-Raid](https://www.mdsec.co.uk/2020/02/iis-raid-backdooring-iis-using-native-modules/).
A successful execution will install a module into IIS using New-WebGlobalModule.
[Managing IIS Modules with PowerShell](https://learn.microsoft.com/en-us/powershell/module/webadministration/set-webglobalmodule?view=windowsserver2022-ps)
[IIS Modules](https://www.microsoft.com/en-us/security/blog/2022/12/12/iis-modules-the-evolution-of-web-shells-and-how-to-detect-them/)

**Supported Platforms:** windows
#### Dependencies:  Run with `powershell`!
##### Description: IIS must be installed in order to add a module to IIS.

##### Check Prereq Commands:
```powershell
$service = get-service w3svc -ErrorAction SilentlyContinue
if($service){ Write-Host "IIS installed on $env:computername" } else { Write-Host "IIS is not installed on $env:computername" } 

```
##### Get Prereq Commands:
```powershell
Install IIS to continue.

```

In [ ]:
Invoke-AtomicTest T1505.004 -TestNumbers 2 -GetPreReqs

#### Attack Commands: Run with `powershell`
```powershell
New-WebGlobalModule -Name DefaultDocumentModule_Atomic -Image %windir%\system32\inetsrv\defdoc.dll
```

In [ ]:
Invoke-AtomicTest T1505.004 -TestNumbers 2

#### Cleanup: 
```powershell
Remove-WebGlobalModule -Name DefaultDocumentModule_Atomic
```

In [ ]:
Invoke-AtomicTest T1505.004 -TestNumbers 2 -Cleanup

## Detection
Monitor for creation and/or modification of files (especially DLLs on webservers) that could be abused as malicious ISAPI extensions/filters or IIS modules. Changes to <code>%windir%\system32\inetsrv\config\applicationhost.config</code> could indicate an IIS module installation.(Citation: Microsoft IIS Modules Overview 2007)(Citation: ESET IIS Malware 2021)

Monitor execution and command-line arguments of <code>AppCmd.exe</code>, which may be abused to install malicious IIS modules.(Citation: Microsoft IIS Modules Overview 2007)(Citation: Unit 42 RGDoor Jan 2018)(Citation: ESET IIS Malware 2021)